In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # para visualizar todas las columnas de los DataFrames

# Gestión de warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Métodos estadísticos
# -----------------------------------------------------------------------
from scipy.stats import zscore  # para calcular el z-score
from sklearn.neighbors import LocalOutlierFactor  # para detectar outliers usando el método LOF
from sklearn.ensemble import IsolationForest  # para detectar outliers usando el método IF
from sklearn.neighbors import NearestNeighbors  # para calcular la epsilon
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler

# Para generar combinaciones de listas
# -----------------------------------------------------------------------
from itertools import product, combinations

# Importación de las clases creadas en nuestro archivo de soporte
# -----------------------------------------------------------------------
from src import soporte_preprocesamiento as sp
from src import soporte_outliers as so
from src import soporte_encoding as se


In [3]:
df = pd.read_csv("datos/datos_limpios.csv", index_col = 0)

In [4]:
df[["rooms", "bathrooms"]] = df[["rooms", "bathrooms"]].astype("str")

In [5]:
columnas_mumericas = df.select_dtypes("number").columns
columnas_categoricas = df.select_dtypes("O").columns

In [7]:
df[columnas_mumericas].info()

<class 'pandas.core.frame.DataFrame'>
Index: 353 entries, 0 to 352
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price      353 non-null    float64
 1   size       353 non-null    float64
 2   latitude   353 non-null    float64
 3   longitude  353 non-null    float64
 4   distance   353 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 16.5 KB


Eliminamos latitud y longitud, pues dado que no vamos a imputar nulos en las categorias numericas, la información de localización que necesitamos nos la dan distnace y municipality, y columnas como latitude y longitude que tienen todos sus valores como unicos carecen de relevancia para el análisis predictivo

In [8]:
df.drop(columns = ["latitude", "longitude"], inplace=True)

In [ ]:
df[columnas_categoricas]